  **To have a fully self-driving car, it is necessary for vehicules to understand and follow traffic signs.**

# This project classify traffic signs present in an image.

We **iterate** over all the classes, **open** image content into an **array** with The PIL library . and append **resized** images and their respective labels in the data and labels list

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image
import os
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

data = []
labels = []
size = (30,30)
classes = 43
for i in range(classes):
    path = "../input/gtsrb-german-traffic-sign/Train/" +str(i)
    images = os.listdir(path)
    for a in images:
        try:
            image = Image.open(path + '/'+ a)
            image = image.resize(size)
            image = np.array(image) /255.0
            data.append(image)
            labels.append(i)
        except:
            print("Error loading image")

convert the list into numpy to feed to the model.
the shape of our data now is (39209, 30, 30, 3)

In [ ]:
data = np.array(data)
labels = np.array(labels)

we split data to training and validation

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.1, random_state=42)
y_train = to_categorical(y_train, 43)
y_test = to_categorical(y_test, 43)

**building and compiling the model + training**

In [ ]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]))
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(43, activation='softmax'))

#Compilation of the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
epochs = 15
history = model.fit(X_train, y_train, batch_size=32, epochs=epochs, validation_data=(X_test, y_test))

Plotting the graph for accuracy and the loss

In [ ]:
plt.figure(0)
plt.plot(history.history['accuracy'], label='training accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

plt.figure(1)
plt.plot(history.history['loss'], label='training loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.title('Loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In a test.csv file, we have the details related to the image path and their respective class labels. we extract images ane their labels and do the same prepocessing as before

In [ ]:
from sklearn.metrics import accuracy_score
y_test = pd.read_csv('/kaggle/input/gtsrb-german-traffic-sign/Test.csv')
labels = y_test["ClassId"].values
imgs = y_test["Path"].values

data=[]
for img in imgs:
    image = Image.open('/kaggle/input/gtsrb-german-traffic-sign/'+img)
    image = image.resize((30,30))
    data.append(np.array(image))
X_test=np.array(data)
pred = model.predict_classes(X_test)
#Accuracy with the test data
from sklearn.metrics import accuracy_score
print(accuracy_score(labels, pred))

In [ ]:
model.save("traffic_classifier.h5")